## montamos Drive

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
path='drive/My Drive/keepcoding/deep_learning/'


## Importar librerías

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Carga de datos e imágenes de test





In [31]:
# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 
import numpy as np
import pandas as pd

test = pd.read_csv(path+'test.csv')
im_test=np.load(path+'im_test.npy', allow_pickle=True)

print(f'Dimensiones del dataset de test: {test.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

Dimensiones del dataset de test: (200, 90)
Dimensiones del dataset de test: (200, 90)


# Preprocesado y data cleaning. Hacemos el mismo que en train

Aquí voy a realizar el datacleaning utilizado en el módulo de machine learning

In [32]:
columns_text=['Host Name','Name','Summary','Space','Description','Neighborhood Overview','Host About','Transit','Access','Interaction','House Rules','Features']
columns_pais=['City','State','Market','Smart Location','Country','Country Code']
columns_neighbourhood=['Street','Neighbourhood','Neighbourhood Cleansed','Neighbourhood Group Cleansed','Zipcode']
columns_pct_high=['Host Acceptance Rate', 'Jurisdiction Names', 'Has Availability',
       'Experiences Offered', 'License', 'Square Feet', 'Monthly Price',
       'Weekly Price', 'Notes']

#vemos porcentaje de nulos en los datos
test_clean=test.replace('none', np.NaN)
test_clean=test_clean.replace('nan', np.NaN)



# eliminamos columnas no relevantes
test_clean = test_clean.drop(test_clean.columns[test_clean.columns.str.contains("Url")], axis=1)
test_clean = test_clean.drop(test_clean.columns[test_clean.columns.str.contains("URL")], axis=1)
test_clean = test_clean.drop(test_clean.columns[test_clean.columns.str.contains("ID")], axis=1)
test_clean = test_clean.drop(test_clean.columns[test_clean.columns.str.contains("Scraped")], axis=1)
test_clean = test_clean.drop(columns_text, axis=1)
test_clean = test_clean.drop(columns_pct_high, axis=1)
# borrar columnas referidas a localizacion
test_clean = test_clean.drop(['Geolocation'], axis=1)
test_clean = test_clean.drop(columns_pais, axis=1)
test_clean = test_clean.drop(['Host Location','Host Neighbourhood'], axis=1)
test_clean = test_clean.drop(['Street','Neighbourhood','Neighbourhood Cleansed','Zipcode'], axis=1)


In [33]:
# fillna 
test_clean['Security Deposit']=test_clean['Security Deposit'].fillna(0)

# rellenar columnas numericas 

col_fillna=['Cleaning Fee','Review Scores Location','Review Scores Value','Review Scores Checkin',
            'Review Scores Accuracy','Review Scores Communication','Review Scores Cleanliness','Review Scores Rating',
           'Reviews per Month','Host Response Rate','Bathrooms','Beds','Bedrooms','Host Total Listings Count','Host Listings Count']

for i in col_fillna:
    test_clean[i].fillna(test_clean[i].mean(),inplace=True)
    
# cambiar columnas con fechas por dias hasta dia de referencia
col_fechas=['Host Since','First Review','Last Review']
dia_ref=pd.to_datetime('2022-01-14')


for i in col_fechas:
    test_clean[i]=(dia_ref-pd.to_datetime(test_clean[i])).dt.days
    test_clean[i]=test_clean[i].fillna(0)
    

In [34]:
#variable host verifications

mode_host_verif=test_clean['Host Verifications'].mode()[0]
test_clean['Host Verifications'].fillna(mode_host_verif,inplace=True)
test_clean['Num Host Verifications']=test_clean['Host Verifications'].str.split(',').str.len()
test_clean = test_clean.drop('Host Verifications', axis=1)

#variable Amenities
mode_amenities=test_clean['Amenities'].mode()[0]

test_clean['Amenities'].fillna(mode_amenities,inplace=True)
test_clean['Num Amenities']=test_clean['Amenities'].str.split(',').str.len()
test_clean = test_clean.drop('Amenities', axis=1)

In [35]:
# property type tiene demasiados valores distintos, resumo los que tienen menos de 10 valores en other
iprop=test_clean['Property Type'].value_counts()<10
iprop=iprop[iprop==True].index
iprop=iprop.insert(1,'Timeshare')
iprop=iprop.insert(1,'Boat')

test_clean.loc[test_clean[test_clean['Property Type'].isin(iprop)].index,['Property Type']]='Other'

In [36]:
# VARIABLEs encoded
col_toencode=['Host Response Time','Neighbourhood Group Cleansed','Property Type','Room Type','Bed Type','Cancellation Policy','Calendar Updated']

test_clean_encoded=test_clean

from sklearn import preprocessing
dict_le={}

for i in col_toencode: 
    le=preprocessing.LabelEncoder()
    le.fit(test_clean_encoded[i])
    dict_le[i]=le
    test_clean_encoded[str(i)+'_encoded']=le.transform(test_clean_encoded[i])
    test_clean_encoded=test_clean_encoded.drop(i,axis=1)
    


In [37]:
test_clean_encoded.shape

(200, 43)

In [38]:
len(test_clean_encoded.columns)

43

# Normalización de datos

In [39]:
y_test = test_clean_encoded['Price'].values    # nos quedamos con price
X_test = test_clean_encoded.drop(['Price'],axis=1).values     # nos quedamos con el resto
feature_names_test = test_clean_encoded.drop(['Price'],axis=1).columns


In [40]:
from pickle import load
XScalerFit = load(open(path+'XScalerFit.pkl', 'rb'))
yScalerFit = load(open(path+'yScalerFit.pkl', 'rb'))

In [43]:
# Generating the standardized values of X and y
X_test=XScalerFit.transform(X_test)
y_test=y_test.reshape(-1, 1)
y_test=yScalerFit.transform(y_test)


In [45]:
print(f'Dimensiones del dataset datos de test: {X_test.shape}')
print(f'Dimensiones del dataset imagenes de test: {im_test.shape}')
print(f'Dimensiones del dataset de target de test: {y_test.shape}')


Dimensiones del dataset datos de test: (200, 42)
Dimensiones del dataset imagenes de test: (200, 224, 224, 3)
Dimensiones del dataset de target de test: (200, 1)


In [46]:
# importamos los paquetes necesarios
from tensorflow.keras import optimizers, Model
from tensorflow.keras.applications import InceptionV3, Xception, VGG16, VGG19, ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import numpy as np
import urllib
import cv2
import matplotlib.pyplot as plt
import imageio as io

input_shape = (224, 224, 3)

# resize test set
im_test_resized = []
for img in im_test:
  im_test_resized.append(np.resize(img, input_shape) / 255)
  
im_test_resized = np.array(im_test_resized)
print(im_test_resized.shape)


(200, 224, 224, 3)


# Cargamos modelos y hacemos predicciones

In [89]:
resultados_train = pd.read_csv(path+'modelos/resultados_train.csv')
resultados_train=resultados_train[['Modelo','RMSE','MAE','MAPE','nombre_modelo']]
resultados_train = resultados_train.drop_duplicates(['Modelo'],
                        keep='first')
resultados_train

,Modelo,RMSE,MAE,MAPE,nombre_modelo
0,ANN,46.936785,20.264633,0.382584,model_ANN.h5
1,ANN_opt,40.441463,22.174349,0.307909,model_ANN_opt.h5
2,CNN_Xception,66.014050,40.473095,0.821547,model_CNN_Xception.h5
3,CNN_opt,60.399190,32.718366,0.668366,model_CNN_opt.h5
4,CNN_vgg16,60.399190,32.718366,0.668366,model_CNN_vgg16.h5
6,ANN+CNN,60.045658,32.580289,0.648674,model_ANN+CNN.h5
7,ANN+CNN_vgg16,70.874517,43.470723,2.286409,model_ANN+CNN_vgg16.h5


In [90]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

models_1d=['ANN','ANN_opt']
models_2d=['CNN_vgg16','CNN_Xception','CNN_opt']
models_comb=['ANN+CNN','ANN+CNN_vgg16']


modelo_full=[]
rmse_full=[]
mae_full=[]
mape_full=[]


from tensorflow import keras

for index,row in resultados_train.iterrows():
  model = keras.models.load_model(path+'/modelos/'+str(row['nombre_modelo']))
  print(f'Vamos a hacer predicción con modelo:'+str(row['Modelo']))
  if row['Modelo'] in models_1d:
    print('Utilizo datos 1d')
    y_pred=model.predict(X_test)
  elif row['Modelo'] in models_2d:
    print('Utilizo datos imagenes')
    y_pred=model.predict(im_test_resized)
  else:
    print('Utilizo datos 1d e imagenes')
    y_pred=model.predict([X_test,im_test_resized])

  y_pred_original=yScalerFit.inverse_transform(y_pred)
  y_test_original=yScalerFit.transform(y_test)
  # calculamos errores
  rmse = math.sqrt(mean_squared_error(y_pred_original, y_test_original))
  mae = mean_absolute_error(y_pred_original, y_test_original)
  mape = mean_absolute_percentage_error(y_pred_original, y_test_original)
  print(mape)
  
  modelo_full.append(row['Modelo'])
  rmse_full.append(rmse)
  mae_full.append(mae)
  mape_full.append(mape)




Vamos a hacer predicción con modelo:ANN
Utilizo datos 1d
7/7 [==============================] - 0s 2ms/step
0.9995983457596508
Vamos a hacer predicción con modelo:ANN_opt
Utilizo datos 1d
7/7 [==============================] - 0s 3ms/step
0.9999878822721884
Vamos a hacer predicción con modelo:CNN_Xception
Utilizo datos imagenes
7/7 [==============================] - 46s 6s/step
1.0002429038100968
Vamos a hacer predicción con modelo:CNN_opt
Utilizo datos imagenes
7/7 [==============================] - 124s 17s/step
1.000242764702388
Vamos a hacer predicción con modelo:CNN_vgg16
Utilizo datos imagenes
7/7 [==============================] - 123s 17s/step
1.000206955971184
Vamos a hacer predicción con modelo:ANN+CNN
Utilizo datos 1d e imagenes
7/7 [==============================] - 14s 2s/step
1.0002352945434327
Vamos a hacer predicción con modelo:ANN+CNN_vgg16
Utilizo datos 1d e imagenes
7/7 [==============================] - 123s 17s/step
0.9999931020962429


In [91]:
results=pd.DataFrame()
results['Modelo']=modelo_full
results['rmse']=rmse_full
results['mae']=mae_full
results['mape']=mape_full
results

,Modelo,rmse,mae,mape
0,ANN,34.763534,33.111160,0.999598
1,ANN_opt,977.798665,976.886745,0.999988
2,CNN_Xception,73.251406,66.691773,1.000243
3,CNN_opt,48.891992,48.823791,1.000243
4,CNN_vgg16,60.454193,59.324035,1.000207
5,ANN+CNN,50.237801,50.237801,1.000235
6,ANN+CNN_vgg16,1714.108647,1713.846064,0.999993


En vista a los resultados escojo el modelo más sencillo la red neuronal porque tiene menor error y es más facil de entrenar. 
En el caso de los otros modelos, creo que entenandolos más o con mayor número de muestras podrían dar resultados mucho mejores.